In [1]:
# === CNT Correlates Audit — One-Cell Runner (paste me into Jupyter) ===
# Scans your CNT_Lab for logs (cooling, EEG laterality, Kuramoto/Ising, cosmology triage),
# tests 8 candidate correlates, and saves a compact report (CSV + TXT).
#
# Outputs:
#   - ./cnt_correlates_report_YYYYMMDD-HHMMSS.csv
#   - ./cnt_correlates_report_YYYYMMDD-HHMMSS.txt
#
# Tips:
#   - Adjust ROOT_DIRS below if your folders differ.
#   - The cell is robust to missing data; it marks unavailable checks as "grey".
#   - No internet needed. Uses numpy/pandas/(scipy if present).
import os, re, glob, json, math, datetime as dt
from pathlib import Path
import numpy as np
import pandas as pd

TS = dt.datetime.now().strftime("%Y%m%d-%H%M%S")

# Try SciPy; if absent, we fall back to a rank-corr helper.
try:
    from scipy.stats import spearmanr, linregress
    HAVE_SCIPY = True
except Exception:
    HAVE_SCIPY = False
    def _rankdata(a):
        # simple average-rank
        a = np.asarray(a, float)
        n = a.size
        order = a.argsort()
        ranks = np.empty(n, float)
        ranks[order] = np.arange(1, n+1)
        # tie handling (average)
        _, inv, counts = np.unique(a, return_inverse=True, return_counts=True)
        sums = np.bincount(inv, ranks)
        avg = sums[counts.cumsum()-counts]/counts
        return avg[inv]
    def spearmanr(x, y, nan_policy="omit"):
        x = np.asarray(x, float); y = np.asarray(y, float)
        mask = np.isfinite(x) & np.isfinite(y)
        if mask.sum() < 3: return np.nan, np.nan
        xr = _rankdata(x[mask]); yr = _rankdata(y[mask])
        r = np.corrcoef(xr, yr)[0,1]
        return float(r), np.nan
    def linregress(x, y):
        x = np.asarray(x, float); y = np.asarray(y, float)
        mask = np.isfinite(x) & np.isfinite(y)
        if mask.sum() < 3:
            return type("LR", (), dict(slope=np.nan, intercept=np.nan, rvalue=np.nan, pvalue=np.nan))()
        A = np.vstack([x[mask], np.ones(mask.sum())]).T
        m, b = np.linalg.lstsq(A, y[mask], rcond=None)[0]
        r = np.corrcoef(x[mask], y[mask])[0,1]
        return type("LR", (), dict(slope=float(m), intercept=float(b), rvalue=float(r), pvalue=np.nan))()

# --------- Configure your roots here ----------
ROOT_DIRS = [
    r"C:\Users\caleb\CNT_Lab",
    r"C:\Users\caleb\cnt_genome",
    r"C:\Users\caleb\Desktop\CNT_Lab",
]

PATTERNS = {
    "cooling_logs": [
        "**/notebooks/archive/*cooling*.csv",
        "**/archive/*cooling*.csv",
        "**/*unified_cooling*.csv",
        "**/cooling/*.csv",
    ],
    "cosmo_triage": [
        "**/cnt_ch_slope_triage/**/ch_triage_timeseries.csv",
        "**/*triage*timeseries*.csv",
    ],
    "eeg_lap_erd": [
        "**/pli_humans_100plus/**/tables/*lap_erd*.csv",
        "**/eeg*/**/lap_*erd*.csv",
    ],
    "kuramoto_ising": [
        "**/cnt_mega_out/**/results*.json",
        "**/cnt_mega_out/**/summary*.json",
        "**/cnt_mega_out/**/kuramoto*.csv",
        "**/cnt_mega_out/**/ising*.csv",
        "**/artifacts/**/metrics/*ising*.csv",
        "**/artifacts/**/metrics/*kuramoto*.csv",
    ],
    "grayscott_images": [
        "**/cnt_mega_out/**/topo_edge_grayscott.png",
        "**/artifacts/**/figures/**/grayscott*.png",
        "**/artifacts/**/figures/**/gray-scott*.png",
    ],
    "logs_text": [
        "**/notebooks/**/*.txt",
        "**/logs/**/*.log",
        "**/*.log",
        "**/*.txt",
    ],
    "glyph_labels": [
        "**/artifacts/**/glyph_labels*.csv",
        "**/glyphs/**/labels*.csv",
    ],
}

def find_files(root_dirs, patterns, limit=None):
    out = []
    for root in root_dirs:
        if not os.path.exists(root): continue
        for pat in patterns:
            out.extend(glob.glob(os.path.join(root, pat), recursive=True))
    out = sorted(set(out))
    return out[:limit] if limit else out

def safe_read_csv(p):
    try:
        return pd.read_csv(p)
    except Exception:
        try:
            return pd.read_csv(p, sep="\t")
        except Exception:
            return None

def slope(x, y):
    x = np.asarray(x); y = np.asarray(y)
    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 3: return np.nan, np.nan, np.nan, np.nan
    res = linregress(x[m], y[m])
    return res.slope, res.intercept, res.rvalue, res.pvalue

def verdict(stat, good="high", thr_green=0.5, thr_yellow=0.25):
    if not np.isfinite(stat): return "grey"
    s = float(stat)
    if good == "low": s = -s
    if s >= thr_green: return "green"
    if s >= thr_yellow: return "yellow"
    return "red"

def row(claim, status, evidence, notes):
    return {"claim": claim, "status": status, "evidence": evidence, "notes": notes}

RUN_LOG = []
def log(msg): RUN_LOG.append(msg)

# 1) Cooling: oscillation amplitude vs clock variance
def analyze_cooling(files):
    rows, used = [], []
    for f in files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        cols = {c.lower(): c for c in df.columns}
        temp_col  = next((cols[c] for c in cols if re.search(r"\btemp|gpu[_ ]?temp", c)), None)
        clock_col = next((cols[c] for c in cols if re.search(r"\bclock|gpu[_ ]?clock", c)), None)
        if temp_col is None or clock_col is None: continue
        t = pd.to_numeric(df[temp_col], errors="coerce").values
        clk = pd.to_numeric(df[clock_col], errors="coerce").values
        if np.isfinite(t).sum()<30 or np.isfinite(clk).sum()<30: continue
        osc = float(np.nanpercentile(t,95) - np.nanpercentile(t,5))
        vclk = float(np.nanvar(clk / (np.nanmedian(clk)+1e-9)))
        rows.append((osc, vclk, f)); used.append(f)
    if not rows:
        return row("Cooling: Temp oscillation amplitude predicts clock stability",
                   "grey","Need logs with GPU temp + GPU clock columns.","Add both fields to your CSVs."), used
    data = pd.DataFrame(rows, columns=["osc_amp","clock_var","file"])
    rho, p = spearmanr(data["osc_amp"], data["clock_var"], nan_policy="omit")
    effect = -rho if np.isfinite(rho) else np.nan
    return row("Cooling: Temp oscillation amplitude predicts clock stability",
               verdict(effect,"high",0.5,0.2),
               f"Spearman ρ={rho:.3f} (p={p:.2g}); N={len(data)}.",
               "Negative ρ supports the claim; sign inverted for scoring."), used

# 2) EEG laterality significance
def analyze_eeg_laterality(files):
    dfs, used = [], []
    for f in files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        if not any("laterality" in c.lower() for c in df.columns): continue
        dfs.append(df); used.append(f)
    if not dfs:
        return row("EEG: Laterality (|μ|/|β|) robustly separates motor vs rest",
                   "grey","No lap_erd laterality tables found.","Export lap_erd_subject*.csv."), used
    df = pd.concat(dfs, ignore_index=True)
    p_cols = [c for c in df.columns if re.fullmatch(r"p|pval|p_value|p-value", c, flags=re.I)]
    strong_rate = np.nan
    if p_cols:
        pvals = pd.to_numeric(pd.concat([df[c] for c in p_cols], axis=0), errors="coerce").dropna().values
        if pvals.size: strong_rate = float((pvals < 1e-4).mean())
    return row("EEG: Laterality (|μ|/|β|) robustly separates motor vs rest",
               verdict(strong_rate,"high",0.6,0.3),
               f"Strong p<1e-4 rate = {strong_rate:.2f}" if np.isfinite(strong_rate) else "No p-value columns found.",
               "High strong-hit rate indicates robust laterality."), used

# 3) Cosmology triage: T-stat vs expansion proxies
def analyze_cosmo_triage(files):
    used = []
    for f in files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        used.append(f)
        cols = {c.lower(): c for c in df.columns}
        Tg = cols.get("t_grad") or next((cols[c] for c in cols if "t_grad" in c), None)
        Ts = cols.get("t_spec") or next((cols[c] for c in cols if "t_spec" in c), None)
        ac = cols.get("a_corr") or next((cols[c] for c in cols if "a_corr" in c), None)
        ap = cols.get("a_peak") or next((cols[c] for c in cols if "a_peak" in c), None)
        al = cols.get("a_len")  or next((cols[c] for c in cols if "a_len"  in c), None)
        if not ((Tg or Ts) and (ac or ap or al)): continue
        pairs = []
        if Tg and ac: s,_,_,_ = slope(df[Tg], df[ac]); pairs.append(("T_grad vs a_corr", s))
        if Ts and ac: s,_,_,_ = slope(df[Ts], df[ac]); pairs.append(("T_spec vs a_corr", s))
        if Tg and ap: s,_,_,_ = slope(df[Tg], df[ap]); pairs.append(("T_grad vs a_peak", s))
        if Ts and ap: s,_,_,_ = slope(df[Ts], df[ap]); pairs.append(("T_spec vs a_peak", s))
        if Tg and al: s,_,_,_ = slope(df[Tg], df[al]); pairs.append(("T_grad vs a_len",  s))
        if Ts and al: s,_,_,_ = slope(df[Ts], df[al]); pairs.append(("T_spec vs a_len",  s))
        if not pairs: continue
        names, slopes_ = zip(*pairs)
        abs_slopes = np.abs(np.array(slopes_))
        try:
            rank_ap = min([i for i,n in enumerate(names) if "a_peak" in n], default=np.nan)
            rank_al = min([i for i,n in enumerate(names) if "a_len"  in n], default=np.nan)
            rank_ac = min([i for i,n in enumerate(names) if "a_corr" in n], default=np.nan)
            ranks = [r for r in [rank_ac, rank_al] if np.isfinite(r)]
            score = float(np.mean([r - rank_ap for r in ranks])) if np.isfinite(rank_ap) and ranks else np.nan
        except Exception:
            score = np.nan
        return row("Cosmo triage: T-statistics favor a_peak over a_corr/a_len",
                   verdict(score,"high",0.7,0.3),
                   f"'a_peak' rank-advantage score = {score:.2f} from {len(pairs)} regressions.",
                   "Higher means T-statistics better explain a_peak."), used
    return row("Cosmo triage: T-statistics favor a_peak over a_corr/a_len",
               "grey","No triage timeseries found.","Save ch_triage_timeseries.csv."), used

# 4) Cross-model echo: Kuramoto ↔ Ising
def analyze_kuramoto_ising(files):
    used, kur_rows, isi_rows = [], [], []
    for f in files:
        if f.lower().endswith(".json"):
            try:
                js = json.loads(Path(f).read_text())
            except Exception:
                continue
            used.append(f)
            Kc = js.get("Kuramoto", {}).get("Kc_est", np.nan)
            bf = js.get("Kuramoto", {}).get("beta_fit", np.nan)
            if np.isfinite(Kc) or np.isfinite(bf): kur_rows.append((Kc, bf, f))
            if "Ising_FSS" in js:
                xw = js["Ising_FSS"].get("crossing_spread", np.nan)
                bnu = js["Ising_FSS"].get("beta_over_nu", np.nan)
                isi_rows.append((xw, bnu, f))
        elif f.lower().endswith(".csv"):
            df = safe_read_csv(f)
            if df is None or df.empty: continue
            used.append(f)
            cols = {c.lower(): c for c in df.columns}
            if "beta_fit" in cols or "kc_est" in cols:
                Kc = pd.to_numeric(df.get(cols.get("kc_est","kc_est"), pd.Series([np.nan])), errors="coerce").median()
                bf = pd.to_numeric(df.get(cols.get("beta_fit","beta_fit"), pd.Series([np.nan])), errors="coerce").median()
                kur_rows.append((Kc, bf, f))
            if "crossing_spread" in cols or "beta_over_nu" in cols:
                xw = pd.to_numeric(df.get(cols.get("crossing_spread","crossing_spread"), pd.Series([np.nan])), errors="coerce").median()
                bnu = pd.to_numeric(df.get(cols.get("beta_over_nu","beta_over_nu"), pd.Series([np.nan])), errors="coerce").median()
                isi_rows.append((xw, bnu, f))
    if not kur_rows or not isi_rows:
        return row("Cross-model echo: Kuramoto onset sharpness predicts Ising FSS crispness",
                   "grey","Need Kuramoto (Kc_est/beta_fit) and Ising (crossing_spread/beta_over_nu) summaries.",
                   "Save json/csv summaries for both."), used
    kur = pd.DataFrame(kur_rows, columns=["Kc_est","beta_fit","file"])
    isi = pd.DataFrame(isi_rows, columns=["crossing_spread","beta_over_nu","file"])
    n = min(len(kur), len(isi))
    df = pd.concat([kur.head(n).reset_index(drop=True),
                    isi.head(n).reset_index(drop=True)], axis=1)
    rho, p = spearmanr(df["beta_fit"], df["crossing_spread"], nan_policy="omit")
    effect = -rho if np.isfinite(rho) else np.nan
    return row("Cross-model echo: Kuramoto onset sharpness predicts Ising FSS crispness",
               verdict(effect,"high",0.5,0.2),
               f"Spearman β_fit vs crossing_spread: ρ={rho:.3f} (p={p:.2g}); N={n}. Lower β_fit ↔ crisper crossings.",
               "Negative ρ supports portability of 'crispness' across models."), used

# 5) Gray-Scott morphology ↔ Kuramoto dispersion (placeholder if inputs exist)
def analyze_grayscott_kuramoto(gs_images, kfiles):
    used = []
    if not gs_images:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "grey","No Gray-Scott edge images found.","Export topo_edge_grayscott.png."), used
    disp_csvs = [f for f in kfiles if f.lower().endswith(".csv") and "kuramoto" in os.path.basename(f).lower()]
    if not disp_csvs:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "grey","No Kuramoto dispersion CSV found.","Export seed,K,R near K≈Kc."), used
    used.extend(gs_images[:1] + disp_csvs[:1])
    return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
               "yellow","Inputs detected; run detailed edge-density/homology in extended notebook.",
               "Compute Canny/homology edge density and regress vs σ(R) near K≈Kc."), used

# 6) Control economics: step size ↔ ΔT per energy
def analyze_cooling_econ(files):
    used, rows = [], []
    for f in files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        cols = {c.lower(): c for c in df.columns}
        temp_col  = next((cols[c] for c in cols if re.search(r"\btemp|gpu[_ ]?temp", c)), None)
        power_col = next((cols[c] for c in cols if re.search(r"\bpower|watts|pwr", c)), None)
        step_col  = next((cols[c] for c in cols if re.search(r"\bstep|schedule|mode", c)), None)
        if temp_col is None or power_col is None: continue
        used.append(f)
        t   = pd.to_numeric(df[temp_col], errors="coerce")
        pwr = pd.to_numeric(df[power_col], errors="coerce")
        if t.notna().sum()<10 or pwr.notna().sum()<10: continue
        dT = float(np.nanpercentile(t,50) - np.nanpercentile(t,95))  # negative if cooling
        E  = float(np.nansum(pwr))  # watt-samples (assumes constant dt)
        eff = -dT/(E+1e-9)          # higher = better cooling per energy
        stepiness = np.nan
        if step_col is not None:
            try:
                stepiness = float(df.shape[0] / (df[step_col].nunique() + 1e-9))
            except Exception:
                stepiness = np.nan
        rows.append((eff, stepiness))
    if not rows:
        return row("Cooling economics: smaller, frequent steps yield better ΔT per energy",
                   "grey","Need cooling logs with power + step/schedule labels.","Log W and step labels."), used
    arr = np.array(rows, float)
    rho, p = spearmanr(arr[:,0], arr[:,1], nan_policy="omit")
    return row("Cooling economics: smaller, frequent steps yield better ΔT per energy",
               verdict(rho,"high",0.4,0.2),
               f"Spearman ρ={rho:.3f} (p={p:.2g}); N={len(arr)}.",
               "Positive ρ supports the claim."), used

# 7) Brittleness sentinel: ConstantInputWarning ↔ high-|ρ| windows
def analyze_brittleness(log_files, metric_csvs):
    used, warnings = [], []
    for f in log_files:
        try:
            txt = Path(f).read_text(errors="ignore")
            if "ConstantInputWarning" in txt:
                warnings.append(f); used.append(f)
        except Exception:
            pass
    if not warnings:
        return row("Brittleness sentinel: constant-input warnings forecast spurious correlations",
                   "grey","No ConstantInputWarning logs found.","If they appear, we’ll cross-check."), used
    high, total = 0, 0
    for f in metric_csvs[:10]:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        corr_cols = [c for c in df.columns if "corr" in c.lower()]
        for c in corr_cols:
            s = pd.to_numeric(df[c], errors="coerce").dropna()
            total += len(s)
            high  += int((np.abs(s) > 0.95).sum())
    rate = (high/total) if total else np.nan
    return row("Brittleness sentinel: constant-input warnings forecast spurious correlations",
               verdict(rate,"low",0.05,0.15),
               f"High-ρ window rate near warnings ≈ {rate:.2f}.",
               "Lower is safer; treat nearby 'discoveries' with caution."), used

# 8) EEG laterality magnitude ↔ glyph label stability (placeholder)
def analyze_laterality_vs_glyph(eeg_files, glyph_label_files):
    used = []
    if not eeg_files or not glyph_label_files:
        return row("EEG laterality magnitude predicts glyph label stability",
                   "grey","Need laterality tables + glyph label time-series.",
                   "Export per-session glyph labels (time windows)."), used
    used.extend(eeg_files[:1] + glyph_label_files[:1])
    return row("EEG laterality magnitude predicts glyph label stability",
               "yellow","Inputs detected; join on session_id to compute flip-rate vs laterality.",
               "Regress flip-rate on |μ|/|β| laterality with SNR covariates."), used

# --------- Run all ----------
def run_all():
    results = []
    used = set()

    cooling = find_files(ROOT_DIRS, PATTERNS["cooling_logs"])
    cosmo   = find_files(ROOT_DIRS, PATTERNS["cosmo_triage"])
    eeg     = find_files(ROOT_DIRS, PATTERNS["eeg_lap_erd"])
    ki      = find_files(ROOT_DIRS, PATTERNS["kuramoto_ising"])
    gs      = find_files(ROOT_DIRS, PATTERNS["grayscott_images"])
    logs    = find_files(ROOT_DIRS, PATTERNS["logs_text"], limit=50)
    glyphs  = find_files(ROOT_DIRS, PATTERNS["glyph_labels"])

    log(f"Found {len(cooling)} cooling logs")
    log(f"Found {len(cosmo)} cosmology triage tables")
    log(f"Found {len(eeg)} EEG laterality tables")
    log(f"Found {len(ki)} Kuramoto/Ising summaries")
    log(f"Found {len(gs)} Gray-Scott edge images")
    log(f"Scanned {len(logs)} text/log files")
    log(f"Found {len(glyphs)} glyph label tables")

    for r,u in [
        analyze_cooling(cooling),
        analyze_eeg_laterality(eeg),
        analyze_cosmo_triage(cosmo),
        analyze_kuramoto_ising(ki),
        analyze_grayscott_kuramoto(gs, ki),
        analyze_cooling_econ(cooling),
        analyze_brittleness(logs, ki + eeg + cosmo + cooling),
        analyze_laterality_vs_glyph(eeg, glyphs),
    ]:
        results.append(r); used.update(u)

    df = pd.DataFrame(results, columns=["claim","status","evidence","notes"])
    csv_path = f"./cnt_correlates_report_{TS}.csv"
    txt_path = f"./cnt_correlates_report_{TS}.txt"
    df.to_csv(csv_path, index=False)

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("== CNT Correlates Audit ==\n")
        f.write(f"Timestamp: {TS}\n\n")
        for k,v in [
            ("Cooling logs", len(cooling)), ("Cosmo triage tables", len(cosmo)),
            ("EEG laterality tables", len(eeg)), ("Kuramoto/Ising summaries", len(ki)),
            ("Gray-Scott edge images", len(gs)), ("Scanned logs/text", len(logs)),
            ("Glyph label tables", len(glyphs)),
        ]:
            f.write(f"- {k}: {v}\n")
        f.write("\n== Results ==\n")
        for _,rowd in df.iterrows():
            f.write(f"[{rowd['status'].upper()}] {rowd['claim']}\n  {rowd['evidence']}\n  {rowd['notes']}\n\n")
        if used:
            f.write("== Used Files (subset) ==\n")
            for p in list(sorted(used))[:50]:
                f.write(f"- {p}\n")

    print("\n".join(RUN_LOG))
    print("\nSaved:")
    print(" -", csv_path)
    print(" -", txt_path)

run_all()
# === end cell ===


Found 8 cooling logs
Found 1 cosmology triage tables
Found 2 EEG laterality tables
Found 0 Kuramoto/Ising summaries
Found 1 Gray-Scott edge images
Scanned 50 text/log files
Found 0 glyph label tables

Saved:
 - ./cnt_correlates_report_20251015-163558.csv
 - ./cnt_correlates_report_20251015-163558.txt


C:\Users\caleb\AppData\Local\Temp\ipykernel_17252\1189349130.py:319: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, p = spearmanr(arr[:,0], arr[:,1], nan_policy="omit")


In [2]:
# === CNT Correlates Audit — Fused Update (paste into Jupyter and run) ===
# What this cell does:
#  • Scans your CNT_Lab for logs (cooling, EEG, cosmology triage, Kuramoto/Ising, Gray–Scott images).
#  • Computes 8 candidate correlates with safe guards and better features.
#  • Adds: safe Spearman; robust "stepiness"; Gray–Scott edge density; Kuramoto dispersion support.
#  • Saves a verdict table (CSV + TXT) next to your notebook.

import os, re, glob, json, math, warnings, datetime as dt
from pathlib import Path
import numpy as np
import pandas as pd

TS = dt.datetime.now().strftime("%Y%m%d-%H%M%S")

# Try SciPy; fall back to in-cell implementations if absent
try:
    from scipy.stats import spearmanr, linregress
    from scipy.stats import ConstantInputWarning
    warnings.filterwarnings("ignore", category=ConstantInputWarning)
    HAVE_SCIPY = True
except Exception:
    HAVE_SCIPY = False
    def _rankdata(a):
        a = np.asarray(a, float)
        order = np.argsort(a)
        ranks = np.empty_like(a, float)
        ranks[order] = np.arange(1, a.size+1)
        # average ties
        vals, inv, cnt = np.unique(a, return_inverse=True, return_counts=True)
        sums = np.bincount(inv, ranks)
        avg = sums[cnt.cumsum()-cnt]/cnt
        return avg[inv]
    def spearmanr(x, y, nan_policy="omit"):
        x = np.asarray(x, float); y = np.asarray(y, float)
        m = np.isfinite(x) & np.isfinite(y)
        if m.sum() < 3: return np.nan, np.nan
        xr = _rankdata(x[m]); yr = _rankdata(y[m])
        r = float(np.corrcoef(xr, yr)[0,1])
        return r, np.nan
    def linregress(x, y):
        x = np.asarray(x, float); y = np.asarray(y, float)
        m = np.isfinite(x) & np.isfinite(y)
        if m.sum() < 3:
            return type("LR", (), dict(slope=np.nan, intercept=np.nan, rvalue=np.nan, pvalue=np.nan))()
        A = np.vstack([x[m], np.ones(m.sum())]).T
        m_, b_ = np.linalg.lstsq(A, y[m], rcond=None)[0]
        r = float(np.corrcoef(x[m], y[m])[0,1])
        return type("LR", (), dict(slope=float(m_), intercept=float(b_), rvalue=r, pvalue=np.nan))()

# ---------- Config ----------
ROOT_DIRS = [
    r"C:\Users\caleb\CNT_Lab",
    r"C:\Users\caleb\cnt_genome",
    r"C:\Users\caleb\Desktop\CNT_Lab",
]

PATTERNS = {
    "cooling_logs": [
        "**/notebooks/archive/*cooling*.csv",
        "**/archive/*cooling*.csv",
        "**/*unified_cooling*.csv",
        "**/cooling/*.csv",
    ],
    "cosmo_triage": [
        "**/cnt_ch_slope_triage/**/ch_triage_timeseries.csv",
        "**/*triage*timeseries*.csv",
    ],
    "eeg_lap_erd": [
        "**/pli_humans_100plus/**/tables/*lap_erd*.csv",
        "**/eeg*/**/lap_*erd*.csv",
    ],
    "kuramoto_ising": [
        "**/cnt_mega_out/**/results*.json",
        "**/cnt_mega_out/**/summary*.json",
        "**/cnt_mega_out/**/kuramoto*.csv",
        "**/cnt_mega_out/**/ising*.csv",
        "**/artifacts/**/metrics/*ising*.csv",
        "**/artifacts/**/metrics/*kuramoto*.csv",
    ],
    "grayscott_images": [
        "**/cnt_mega_out/**/topo_edge_grayscott.png",
        "**/artifacts/**/figures/**/grayscott*.png",
        "**/artifacts/**/figures/**/gray-scott*.png",
    ],
    "logs_text": [
        "**/notebooks/**/*.txt",
        "**/logs/**/*.log",
        "**/*.log",
        "**/*.txt",
    ],
    "glyph_labels": [
        "**/artifacts/**/glyph_labels*.csv",
        "**/glyphs/**/labels*.csv",
    ],
    # NEW: optional Kuramoto dispersion CSV (seed,K,R) near K≈Kc
    "kuramoto_disp": [
        "**/cnt_mega_out/**/kuramoto_dispersion*.csv",
        "**/artifacts/**/metrics/**/kuramoto_dispersion*.csv",
    ]
}

# ---------- Helpers ----------
RUN_LOG = []
def log(m): 
    RUN_LOG.append(m)

def find_files(root_dirs, patterns, limit=None):
    out = []
    for root in root_dirs:
        if not os.path.exists(root): continue
        for pat in patterns:
            out.extend(glob.glob(os.path.join(root, pat), recursive=True))
    out = sorted(set(out))
    return out[:limit] if limit else out

def safe_read_csv(p):
    for sep in [",", "\t", "|", ";"]:
        try:
            df = pd.read_csv(p, sep=sep)
            if df.shape[1] >= 2: return df
        except Exception:
            continue
    return None

def slope(x, y):
    x = np.asarray(x); y = np.asarray(y)
    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 3: return np.nan, np.nan, np.nan, np.nan
    lr = linregress(x[m], y[m])
    return lr.slope, lr.intercept, lr.rvalue, lr.pvalue

def safe_spearman(x, y):
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    if m.sum() < 3: return np.nan, np.nan
    if np.nanstd(x[m]) == 0 or np.nanstd(y[m]) == 0:
        return np.nan, np.nan
    return spearmanr(x[m], y[m], nan_policy="omit")

def verdict(stat, good="high", thr_green=0.5, thr_yellow=0.25):
    if not np.isfinite(stat): return "grey"
    s = float(stat)
    if good == "low": s = -s
    if s >= thr_green: return "green"
    if s >= thr_yellow: return "yellow"
    return "red"

def row(claim, status, evidence, notes):
    return {"claim": claim, "status": status, "evidence": evidence, "notes": notes}

# --- Image edge density (no extra deps): simple Sobel + percentile threshold ---
def image_edge_density(path, thresh_pct=90):
    try:
        import matplotlib.image as mpimg
        img = mpimg.imread(path)
    except Exception:
        return np.nan
    arr = np.asarray(img)
    if arr.ndim == 3:
        arr = 0.299*arr[...,0] + 0.587*arr[...,1] + 0.114*arr[...,2]
    arr = arr.astype(float)
    # Sobel kernels
    Kx = np.array([[1,0,-1],[2,0,-2],[1,0,-1]], float)
    Ky = Kx.T
    # pad
    pad = np.pad(arr, 1, mode="reflect")
    Gx = (Kx[0,0]*pad[:-2,:-2] + Kx[0,1]*pad[:-2,1:-1] + Kx[0,2]*pad[:-2,2:] +
          Kx[1,0]*pad[1:-1,:-2] + Kx[1,1]*pad[1:-1,1:-1] + Kx[1,2]*pad[1:-1,2:] +
          Kx[2,0]*pad[2:,  :-2] + Kx[2,1]*pad[2:,  1:-1] + Kx[2,2]*pad[2:,  2:])
    Gy = (Ky[0,0]*pad[:-2,:-2] + Ky[0,1]*pad[:-2,1:-1] + Ky[0,2]*pad[:-2,2:] +
          Ky[1,0]*pad[1:-1,:-2] + Ky[1,1]*pad[1:-1,1:-1] + Ky[1,2]*pad[1:-1,2:] +
          Ky[2,0]*pad[2:,  :-2] + Ky[2,1]*pad[2:,  1:-1] + Ky[2,2]*pad[2:,  2:])
    mag = np.hypot(Gx, Gy)
    thr = np.nanpercentile(mag, thresh_pct)
    edges = (mag >= thr)
    return float(np.mean(edges))  # fraction of "edge" pixels

# --- Robust "stepiness" detector: uses explicit step/mode if present, else change-points in power/fan ---
def compute_stepiness(df, step_cols=("step","schedule","mode"), change_cols=("power","watts","pwr","fan","target")):
    cols = {c.lower(): c for c in df.columns}
    # 1) explicit step column
    for k in step_cols:
        if any(k in c for c in cols):
            c = [cols[c] for c in cols if k in c][0]
            vals = pd.Series(df[c]).astype(str)
            uniq = vals.nunique()
            if uniq > 1:
                return float(df.shape[0] / uniq)
    # 2) change points on power/fan-like columns
    for k in change_cols:
        hits = [cols[c] for c in cols if k in c]
        for c in hits:
            v = pd.to_numeric(df[c], errors="coerce")
            if v.notna().sum() < 5: continue
            # count big jumps (>= 5% of range) as "steps"
            rng = np.nanpercentile(v, 95) - np.nanpercentile(v, 5)
            if rng <= 0: continue
            jumps = np.abs(np.diff(v.fillna(method="ffill").fillna(method="bfill").values))
            steps = int((jumps >= 0.05 * rng).sum())
            if steps > 0:
                return float(df.shape[0] / max(1, steps))
    return np.nan

# ---------- 1) Cooling: oscillation amplitude vs clock variance ----------
def analyze_cooling(cooling_files):
    rows, used = [], []
    for f in cooling_files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        cols = {c.lower(): c for c in df.columns}
        temp_col  = next((cols[c] for c in cols if re.search(r"\btemp|gpu[_ ]?temp", c)), None)
        clock_col = next((cols[c] for c in cols if re.search(r"\bclock|gpu[_ ]?clock", c)), None)
        if temp_col is None or clock_col is None: continue
        t = pd.to_numeric(df[temp_col], errors="coerce").values
        clk = pd.to_numeric(df[clock_col], errors="coerce").values
        if np.isfinite(t).sum()<30 or np.isfinite(clk).sum()<30: continue
        osc = float(np.nanpercentile(t,95) - np.nanpercentile(t,5))
        vclk = float(np.nanvar(clk / (np.nanmedian(clk)+1e-9)))
        rows.append((osc, vclk, f)); used.append(f)
    if not rows:
        return row("Cooling: Temp oscillation amplitude predicts clock stability",
                   "grey","Need logs with GPU temp + GPU clock columns.","Add both fields to your CSVs."), used
    data = pd.DataFrame(rows, columns=["osc_amp","clock_var","file"])
    rho, p = safe_spearman(data["osc_amp"], data["clock_var"])
    effect = -rho if np.isfinite(rho) else np.nan
    return row("Cooling: Temp oscillation amplitude predicts clock stability",
               verdict(effect,"high",0.5,0.2),
               f"Spearman ρ={rho:.3f} (p={p:.2g}); N={len(data)}.",
               "Negative ρ supports the claim; sign inverted for scoring."), used

# ---------- 2) EEG laterality significance ----------
def analyze_eeg_laterality(eeg_files):
    dfs, used = [], []
    for f in eeg_files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        if not any("laterality" in c.lower() for c in df.columns): continue
        dfs.append(df); used.append(f)
    if not dfs:
        return row("EEG: Laterality (|μ|/|β|) robustly separates motor vs rest",
                   "grey","No lap_erd laterality tables found.","Export lap_erd_subject*.csv."), used
    df = pd.concat(dfs, ignore_index=True)
    p_cols = [c for c in df.columns if re.fullmatch(r"p|pval|p_value|p-value", c, flags=re.I)]
    strong_rate = np.nan
    if p_cols:
        pvals = pd.to_numeric(pd.concat([df[c] for c in p_cols], axis=0), errors="coerce").dropna().values
        if pvals.size: strong_rate = float((pvals < 1e-4).mean())
    return row("EEG: Laterality (|μ|/|β|) robustly separates motor vs rest",
               verdict(strong_rate,"high",0.6,0.3),
               f"Strong p<1e-4 rate = {strong_rate:.2f}" if np.isfinite(strong_rate) else "No p-value columns found.",
               "High strong-hit rate indicates robust laterality."), used

# ---------- 3) Cosmology triage ----------
def analyze_cosmo_triage(cosmo_files):
    used = []
    for f in cosmo_files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        used.append(f)
        cols = {c.lower(): c for c in df.columns}
        Tg = cols.get("t_grad") or next((cols[c] for c in cols if "t_grad" in c), None)
        Ts = cols.get("t_spec") or next((cols[c] for c in cols if "t_spec" in c), None)
        ac = cols.get("a_corr") or next((cols[c] for c in cols if "a_corr" in c), None)
        ap = cols.get("a_peak") or next((cols[c] for c in cols if "a_peak" in c), None)
        al = cols.get("a_len")  or next((cols[c] for c in cols if "a_len"  in c), None)
        if not ((Tg or Ts) and (ac or ap or al)): continue
        pairs = []
        if Tg and ac: s,_,_,_ = slope(df[Tg], df[ac]); pairs.append(("T_grad vs a_corr", s))
        if Ts and ac: s,_,_,_ = slope(df[Ts], df[ac]); pairs.append(("T_spec vs a_corr", s))
        if Tg and ap: s,_,_,_ = slope(df[Tg], df[ap]); pairs.append(("T_grad vs a_peak", s))
        if Ts and ap: s,_,_,_ = slope(df[Ts], df[ap]); pairs.append(("T_spec vs a_peak", s))
        if Tg and al: s,_,_,_ = slope(df[Tg], df[al]); pairs.append(("T_grad vs a_len",  s))
        if Ts and al: s,_,_,_ = slope(df[Ts], df[al]); pairs.append(("T_spec vs a_len",  s))
        if not pairs: continue
        names, slopes_ = zip(*pairs)
        abs_slopes = np.abs(np.array(slopes_))
        # rank advantage: a_peak should rank above a_corr/a_len
        def first_rank(substr):
            idxs = [i for i,n in enumerate(names) if substr in n]
            return min(idxs) if idxs else np.nan
        rank_ap = first_rank("a_peak"); rank_al = first_rank("a_len"); rank_ac = first_rank("a_corr")
        others = [r for r in [rank_ac, rank_al] if np.isfinite(r)]
        score = float(np.mean([r - rank_ap for r in others])) if np.isfinite(rank_ap) and others else np.nan
        return row("Cosmo triage: T-statistics favor a_peak over a_corr/a_len",
                   verdict(score,"high",0.7,0.3),
                   f"'a_peak' rank-advantage score = {score:.2f} from {len(pairs)} regressions.",
                   "Higher means T-statistics better explain a_peak."), used
    return row("Cosmo triage: T-statistics favor a_peak over a_corr/a_len",
               "grey","No triage timeseries found.","Save ch_triage_timeseries.csv."), used

# ---------- 4) Cross-model echo: Kuramoto ↔ Ising ----------
def analyze_kuramoto_ising(ki_files):
    used, kur_rows, isi_rows = [], [], []
    for f in ki_files:
        if f.lower().endswith(".json"):
            try:
                js = json.loads(Path(f).read_text())
            except Exception:
                continue
            used.append(f)
            Kc = js.get("Kuramoto", {}).get("Kc_est", np.nan)
            bf = js.get("Kuramoto", {}).get("beta_fit", np.nan)
            if np.isfinite(Kc) or np.isfinite(bf): kur_rows.append((Kc, bf, f))
            if "Ising_FSS" in js:
                xw = js["Ising_FSS"].get("crossing_spread", np.nan)
                bnu = js["Ising_FSS"].get("beta_over_nu", np.nan)
                isi_rows.append((xw, bnu, f))
        elif f.lower().endswith(".csv"):
            df = safe_read_csv(f)
            if df is None or df.empty: continue
            used.append(f)
            cols = {c.lower(): c for c in df.columns}
            if "beta_fit" in cols or "kc_est" in cols:
                Kc = pd.to_numeric(df.get(cols.get("kc_est","kc_est"), pd.Series([np.nan])), errors="coerce").median()
                bf = pd.to_numeric(df.get(cols.get("beta_fit","beta_fit"), pd.Series([np.nan])), errors="coerce").median()
                kur_rows.append((Kc, bf, f))
            if "crossing_spread" in cols or "beta_over_nu" in cols:
                xw = pd.to_numeric(df.get(cols.get("crossing_spread","crossing_spread"), pd.Series([np.nan])), errors="coerce").median()
                bnu = pd.to_numeric(df.get(cols.get("beta_over_nu","beta_over_nu"), pd.Series([np.nan])), errors="coerce").median()
                isi_rows.append((xw, bnu, f))
    if not kur_rows or not isi_rows:
        return row("Cross-model echo: Kuramoto onset sharpness predicts Ising FSS crispness",
                   "grey","Need Kuramoto (Kc_est/beta_fit) and Ising (crossing_spread/beta_over_nu) summaries.",
                   "Save json/csv summaries for both."), used
    kur = pd.DataFrame(kur_rows, columns=["Kc_est","beta_fit","file"])
    isi = pd.DataFrame(isi_rows, columns=["crossing_spread","beta_over_nu","file"])
    n = min(len(kur), len(isi))
    df = pd.concat([kur.head(n).reset_index(drop=True),
                    isi.head(n).reset_index(drop=True)], axis=1)
    rho, p = safe_spearman(df["beta_fit"], df["crossing_spread"])
    effect = -rho if np.isfinite(rho) else np.nan
    return row("Cross-model echo: Kuramoto onset sharpness predicts Ising FSS crispness",
               verdict(effect,"high",0.5,0.2),
               f"Spearman β_fit vs crossing_spread: ρ={rho:.3f} (p={p:.2g}); N={n}. Lower β_fit ↔ crisper crossings.",
               "Negative ρ supports portability of 'crispness' across models."), used

# ---------- 5) Morphology→Dynamics: Gray–Scott edge density ↔ Kuramoto dispersion ----------
def analyze_grayscott_kuramoto(gs_images, disp_files):
    used = []
    if not gs_images:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "grey","No Gray-Scott edge images found.","Export topo_edge_grayscott.png."), used
    # Take first image (or aggregate later if you have many)
    ed = image_edge_density(gs_images[0])
    used.append(gs_images[0])

    if not disp_files:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "yellow", f"Edge density ~ {ed:.3f}; no Kuramoto dispersion CSV found.",
                   "Export seed,K,R near K≈Kc as 'kuramoto_dispersion.csv'."), used

    # Compute dispersion metric: std of R in a narrow band around median K
    disp = []
    for f in disp_files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        cols = {c.lower(): c for c in df.columns}
        if not all(k in cols for k in ["seed","k","r"]): 
            # try lowercase fallback names
            need = ["seed","k","r"]; got = list(cols.keys())
            continue
        K = pd.to_numeric(df[cols["k"]], errors="coerce")
        R = pd.to_numeric(df[cols["r"]], errors="coerce")
        if K.notna().sum()<10: continue
        K0 = np.nanmedian(K)
        win = np.abs(K - K0) <= (0.02 * (np.nanpercentile(K,95)-np.nanpercentile(K,5)) + 1e-9)
        sigma_R = float(np.nanstd(R[win]))
        disp.append(sigma_R)
        used.append(f)

    if not disp:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "yellow", f"Edge density ~ {ed:.3f}; dispersion metric not computed (K/R missing or too sparse).",
                   "Ensure columns seed,K,R and sufficient rows near K≈median(K)."), used

    # If you eventually have multiple images+disp files, correlate; with one pair, report metric only.
    if len(disp) == 1:
        return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
                   "yellow", f"Edge density ~ {ed:.3f}; σ(R|K≈Kc) ~ {disp[0]:.4f}.",
                   "Add more runs to correlate edge density vs σ(R)."), used

    # Multi-run: correlate edge density (per image) with dispersion across runs (here we only used first image, but structure supports list)
    # Placeholder association if multiple pairs are available
    return row("Morphology→Dynamics: Gray-Scott edge density predicts Kuramoto dispersion near Kc",
               "yellow", "Multiple runs detected; extend mapping to pair each image with its dispersion.", 
               "Name images/CSVs with matching tags to correlate per-run."), used

# ---------- 6) Control economics: step size ↔ ΔT per energy ----------
def analyze_cooling_econ(cooling_files):
    used, rows = [], []
    for f in cooling_files:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        cols = {c.lower(): c for c in df.columns}
        temp_col  = next((cols[c] for c in cols if re.search(r"\btemp|gpu[_ ]?temp", c)), None)
        power_col = next((cols[c] for c in cols if re.search(r"\bpower|watts|pwr", c)), None)
        if temp_col is None or power_col is None: continue
        used.append(f)
        t   = pd.to_numeric(df[temp_col], errors="coerce")
        pwr = pd.to_numeric(df[power_col], errors="coerce")
        if t.notna().sum()<10 or pwr.notna().sum()<10: continue
        # ΔT per "joule" proxy (assuming constant dt sampling)
        dT = float(np.nanpercentile(t,50) - np.nanpercentile(t,95))  # negative if cooling
        E  = float(np.nansum(pwr))
        eff = -dT/(E+1e-9)          # higher = better cooling per energy
        stepiness = compute_stepiness(df)
        rows.append((eff, stepiness))
    if not rows:
        return row("Cooling economics: smaller, frequent steps yield better ΔT per energy",
                   "grey","Need cooling logs with power + step/schedule/targets.","Log W and step/target labels."), used
    arr = np.array(rows, float)
    rho, p = safe_spearman(arr[:,0], arr[:,1])
    return row("Cooling economics: smaller, frequent steps yield better ΔT per energy",
               verdict(rho,"high",0.4,0.2),
               f"Spearman ρ={rho:.3f} (p={p:.2g}); N={len(arr)}. (auto stepiness detector)",
               "Positive ρ supports the claim; guards avoid constant-input errors."), used

# ---------- 7) Brittleness sentinel ----------
def analyze_brittleness(log_files, metric_csvs):
    used, warnings_hit = [], []
    for f in log_files:
        try:
            txt = Path(f).read_text(errors="ignore")
        except Exception:
            continue
        if "ConstantInputWarning" in txt or "ConstantInput" in txt:
            warnings_hit.append(f); used.append(f)
    if not warnings_hit:
        return row("Brittleness sentinel: constant-input warnings forecast spurious correlations",
                   "grey","No ConstantInputWarning logs found.","If they appear, we’ll cross-check."), used
    high, total = 0, 0
    for f in metric_csvs[:10]:
        df = safe_read_csv(f)
        if df is None or df.empty: continue
        corr_cols = [c for c in df.columns if "corr" in c.lower()]
        for c in corr_cols:
            s = pd.to_numeric(df[c], errors="coerce").dropna()
            total += len(s)
            high  += int((np.abs(s) > 0.95).sum())
    rate = (high/total) if total else np.nan
    return row("Brittleness sentinel: constant-input warnings forecast spurious correlations",
               verdict(rate,"low",0.05,0.15),
               f"High-ρ window rate near warnings ≈ {rate:.2f}.",
               "Lower is safer; treat nearby 'discoveries' with caution."), used

# ---------- 8) Laterality ↔ glyph stability (placeholder until glyph labels exist) ----------
def analyze_laterality_vs_glyph(eeg_files, glyph_label_files):
    used = []
    if not eeg_files or not glyph_label_files:
        return row("EEG laterality magnitude predicts glyph label stability",
                   "grey","Need laterality tables + glyph label time-series.",
                   "Export per-session glyph labels (time windows) with session_id."), used
    used.extend(eeg_files[:1] + glyph_label_files[:1])
    return row("EEG laterality magnitude predicts glyph label stability",
               "yellow","Inputs detected; join on session_id to compute flip-rate vs laterality.",
               "Regress flip-rate on |μ|/|β| laterality with SNR covariates."), used

# ---------- Runner ----------
def run_all():
    results = []
    used = set()

    cooling = find_files(ROOT_DIRS, PATTERNS["cooling_logs"])
    cosmo   = find_files(ROOT_DIRS, PATTERNS["cosmo_triage"])
    eeg     = find_files(ROOT_DIRS, PATTERNS["eeg_lap_erd"])
    ki      = find_files(ROOT_DIRS, PATTERNS["kuramoto_ising"])
    gs      = find_files(ROOT_DIRS, PATTERNS["grayscott_images"])
    logs    = find_files(ROOT_DIRS, PATTERNS["logs_text"], limit=50)
    glyphs  = find_files(ROOT_DIRS, PATTERNS["glyph_labels"])
    kdisp   = find_files(ROOT_DIRS, PATTERNS["kuramoto_disp"])

    log(f"Found {len(cooling)} cooling logs")
    log(f"Found {len(cosmo)} cosmology triage tables")
    log(f"Found {len(eeg)} EEG laterality tables")
    log(f"Found {len(ki)} Kuramoto/Ising summaries")
    log(f"Found {len(gs)} Gray-Scott edge images")
    log(f"Scanned {len(logs)} text/log files")
    log(f"Found {len(glyphs)} glyph label tables")
    log(f"Found {len(kdisp)} Kuramoto dispersion CSVs")

    for r,u in [
        analyze_cooling(cooling),
        analyze_eeg_laterality(eeg),
        analyze_cosmo_triage(cosmo),
        analyze_kuramoto_ising(ki),
        analyze_grayscott_kuramoto(gs, kdisp),
        analyze_cooling_econ(cooling),
        analyze_brittleness(logs, ki + eeg + cosmo + cooling),
        analyze_laterality_vs_glyph(eeg, glyphs),
    ]:
        results.append(r); used.update(u)

    df = pd.DataFrame(results, columns=["claim","status","evidence","notes"])
    csv_path = f"./cnt_correlates_report_{TS}.csv"
    txt_path = f"./cnt_correlates_report_{TS}.txt"
    df.to_csv(csv_path, index=False)

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("== CNT Correlates Audit (Fused) ==\n")
        f.write(f"Timestamp: {TS}\n\n")
        for k,v in [
            ("Cooling logs", len(cooling)), ("Cosmo triage tables", len(cosmo)),
            ("EEG laterality tables", len(eeg)), ("Kuramoto/Ising summaries", len(ki)),
            ("Kuramoto dispersion CSVs", len(kdisp)), ("Gray-Scott edge images", len(gs)),
            ("Scanned logs/text", len(logs)), ("Glyph label tables", len(glyphs)),
        ]:
            f.write(f"- {k}: {v}\n")
        f.write("\n== Results ==\n")
        for _,rowd in df.iterrows():
            f.write(f"[{rowd['status'].upper()}] {rowd['claim']}\n  {rowd['evidence']}\n  {rowd['notes']}\n\n")
        if used:
            f.write("== Used Files (subset) ==\n")
            for p in list(sorted(used))[:50]:
                f.write(f"- {p}\n")

    print("\n".join(RUN_LOG))
    print("\nSaved:")
    print(" -", csv_path)
    print(" -", txt_path)

run_all()
# === end cell ===


Found 8 cooling logs
Found 1 cosmology triage tables
Found 2 EEG laterality tables
Found 0 Kuramoto/Ising summaries
Found 1 Gray-Scott edge images
Scanned 50 text/log files
Found 0 glyph label tables
Found 0 Kuramoto dispersion CSVs

Saved:
 - ./cnt_correlates_report_20251015-164130.csv
 - ./cnt_correlates_report_20251015-164130.txt


C:\Users\caleb\AppData\Local\Temp\ipykernel_17252\462416049.py:198: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  jumps = np.abs(np.diff(v.fillna(method="ffill").fillna(method="bfill").values))
C:\Users\caleb\AppData\Local\Temp\ipykernel_17252\462416049.py:198: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  jumps = np.abs(np.diff(v.fillna(method="ffill").fillna(method="bfill").values))
